In [5]:
# Import modules
import numpy as np

In [6]:
# Import data
d = np.fromfile('data/d.dat')
fP = np.fromfile('data/fP.dat')
fr = np.fromfile('data/fr.dat')
localK = np.fromfile('data/localK.dat')
solution = np.fromfile('data/solution.dat')

In [48]:
APq_array = []

sub_x = 4
sub_y = 3

NP = (sub_x + 1)*(sub_y + 1)

for j in range(sub_y):
    for i in range(sub_x):
        x0 = j*(sub_x + 1) + i 
        x1 = j*(sub_x + 1) + i + 1
        x2 = (j + 1)*(sub_x + 1) + i 
        x3 = (j + 1)*(sub_x + 1) + i + 1
        APqs = np.zeros([NP, 4])
        APqs[x0, 0] = 1
        APqs[x1, 1] = 1
        APqs[x2, 2] = 1
        APqs[x3, 3] = 1
        APq_array.append(APqs)        

In [59]:
Nr_x = 4
Nr_y = 3
ARr_array = []
Nr = Nr_x*Nr_y - 4
NR = Nr*(sub_x*sub_y)

for j in range(sub_y):
    for i in range(sub_x):
        nodes = []
        
        for k in range(Nr_y):
            if (k == 0):
                nodes = np.concatenate([nodes, np.arange(Nr_x - 2) + Nr*sub_x*j + (Nr_x - 2)*i])
            elif (k == Nr_y - 1):
                nodes = np.concatenate([nodes, np.arange(Nr_x - 2) + Nr*sub_x*j + (Nr_x - 2)*i + (Nr_x*k - 2)*sub_x])
            else:
                nodes = np.concatenate([nodes, np.arange(Nr_x) + Nr*sub_x*j + (Nr_x)*i + (Nr_x*k - 2)*sub_x])

        ARrs = np.zeros([NR, len(nodes)])
        for col, node in enumerate(nodes):
            ARrs[int(node), col] = 1
        ARr_array.append(ARrs)

[array([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0